In [5]:
import SimpleITK as sitk
import numpy as np
import os

def resize_segmentation_3d(raw_segmentation, new_shape=(704, 704)):
    """Resize a complete 3D segmentation volume from 1024x1024 to new shape (704, 704), preserving the depth.

    Args:
        raw_segmentation (np.array): 3D segmentation (shape: [num_slices, height, width]).
        new_shape (tuple, optional): Target shape (default is (704, 704)).

    Returns:
        np.array: Resized 3D segmentation (shape: [num_slices, 704, 704]).
    """
    # Convert numpy array to SimpleITK image
    seg_image = sitk.GetImageFromArray(raw_segmentation)

    # Get original spacing
    original_spacing = seg_image.GetSpacing()

    # Compute new spacing to preserve the depth (Z dimension stays the same)
    new_spacing = (
        original_spacing[0] * raw_segmentation.shape[2] / new_shape[0],  # Adjust X spacing
        original_spacing[1] * raw_segmentation.shape[1] / new_shape[1],  # Adjust Y spacing
        original_spacing[2]  # Z spacing stays the same
    )

    # Set up resampler
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize((new_shape[0], new_shape[1], raw_segmentation.shape[0]))  # (width, height, depth)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)  # Nearest neighbor for segmentation
    resampler.SetOutputSpacing(new_spacing)

    # Apply resampling
    resampled_seg = resampler.Execute(seg_image)

    # Convert back to numpy array
    resampled_seg_array = sitk.GetArrayFromImage(resampled_seg)  # (num_slices, 704, 704)

    return resampled_seg_array

def save_resized_segmentation(resized_segmentation, output_path):
    """Save the resized segmentation as a NIfTI file.

    Args:
        resized_segmentation (np.array): Resized 3D segmentation array.
        output_path (str): Path to save the output NIfTI file.
    """
    # Convert numpy array to SimpleITK image
    seg_image = sitk.GetImageFromArray(resized_segmentation)

    # Save to NIfTI format
    sitk.WriteImage(seg_image, output_path)
    print(f"Resized segmentation saved to {output_path}")

# Example usage:
# Load your segmentation data (make sure it's a 3D numpy array)
folder=r'W:\rubenvdw\Dataset\Segmentations_1024'
for file_name in os.listdir(folder):
    if file_name.endswith('.nii.gz'):  # Check if the file is a .nii.gz file
        file_path = os.path.join(folder, file_name)
        print(f"Processing file: {file_path}")
        
        # Read the segmentation file
        raw_segmentation = sitk.GetArrayFromImage(sitk.ReadImage(file_path))

        resized_segmentation = resize_segmentation_3d(raw_segmentation, new_shape=(704, 704))
        
        # Save the resized segmentation with '704' added to the filename
        output_file_name = file_name.replace('.nii.gz', '_704.nii.gz')
        output_path = os.path.join(folder, output_file_name)
        
        save_resized_segmentation(resized_segmentation, output_path)

Processing file: W:\rubenvdw\Dataset\Segmentations_1024\EST-NEMC-0027-RCx_HPNV.nii.gz
Resized segmentation saved to W:\rubenvdw\Dataset\Segmentations_1024\EST-NEMC-0027-RCx_HPNV_704.nii.gz
Processing file: W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0076_HPNV.nii.gz
Resized segmentation saved to W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0076_HPNV_704.nii.gz
Processing file: W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0079_HPNV.nii.gz
Resized segmentation saved to W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0079_HPNV_704.nii.gz
Processing file: W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0084_HPNV.nii.gz
Resized segmentation saved to W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0084_HPNV_704.nii.gz
Processing file: W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0086_HPNV.nii.gz
Resized segmentation saved to W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0086_HPNV_704.nii.gz
Processing file: W:\rubenvdw\Dataset\Segmentations_1024\NLD-ISALA-0090_HPNV.nii

In [3]:
import nibabel as nib
import numpy as np

# Load the images
image1_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Data\\nnUNet_raw\\Dataset501\\imagesTr\\ESTNEMC0013_1_frame0_198_0000.nii'
image2_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Data\\nnUNet_raw\\Dataset905_SegmentOCT3d3\\imagesTr\\ESTNEMC0013_1_frame0_198_0000.nii.gz'
label1_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Data\\nnUNet_raw\\Dataset501\\labelsTr\\ESTNEMC0013_1_frame0_198.nii'
label2_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Data\\nnUNet_raw\\Dataset905_SegmentOCT3d3\\labelsTr\\ESTNEMC0013_1_frame0_198.nii.gz'

image1 = nib.load(image1_path)
image2 = nib.load(image2_path)

# Convert images to numpy arrays
image1_array = image1.get_fdata()
image2_array = image2.get_fdata()

label1 = nib.load(label1_path)
label2 = nib.load(label2_path)

# Convert images to numpy arrays
label1_array = label1.get_fdata()
label2_array = label2.get_fdata()

# Print the shapes of the images
print("Shape of image 1:", image1_array.shape)
print("Shape of image 2:", image2_array.shape)

print("Shape of image 1:", label1_array.shape)
print("Shape of image 2:", label2_array.shape)

Shape of image 1: (704, 704, 7)
Shape of image 2: (704, 704, 1)
Shape of image 1: (704, 704, 1)
Shape of image 2: (704, 704, 1)


In [5]:
import pickle
import numpy as np

# Path to the pickle file
pickle_file_path = 'Z:\\rubenvdw\\nnunetv2\\nnUNet\\nnunetv2\\Predictions\\Dataset905_SegmentOCT3d3\\Predicted_files\\NLDRADB0089_1_frame520_132.pkl'
# Path to the npz file
npz_file_path = pickle_file_path.replace('.pkl', '.npz')

# Load the npz file
npz_data = np.load(npz_file_path)

# Print the loaded data
print(npz_data['probabilities'].shape)

(15, 1, 704, 704)
